In [1]:
import datajoint as dj
import numpy as np
import time

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')

Connecting celiib@10.28.0.34:3306


In [3]:
pinky.ExcitatoryLeftoverMeshes & pinky.CurrentSegmentation & 'decimation_ratio=0.35' & pinky.LeftoverCoarseLabelFinal.proj() & "n_triangles>0"


segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices number of vertices in this mesh pieces that were filtered away,n_triangles number of triangles in this mesh pieces that were filtered away,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices),recovered_perc number of faces of this recovered mesh / number of faces in filtered pymeshfix mesh
3,648518346341371119,0.35,287,489,=BLOB=,=BLOB=,0.00109
3,648518346349386137,0.35,66,100,=BLOB=,=BLOB=,0.00178
3,648518346349470171,0.35,31607,62878,=BLOB=,=BLOB=,0.08902
3,648518346349471156,0.35,2220,4339,=BLOB=,=BLOB=,0.00842
3,648518346349471500,0.35,128,235,=BLOB=,=BLOB=,0.00051
3,648518346349471562,0.35,3969,7687,=BLOB=,=BLOB=,0.00857
3,648518346349471565,0.35,335,543,=BLOB=,=BLOB=,0.00036
3,648518346349471910,0.35,1034,1988,=BLOB=,=BLOB=,0.00151
3,648518346349472574,0.35,649,1208,=BLOB=,=BLOB=,0.00115
3,648518346349472601,0.35,106,199,=BLOB=,=BLOB=,0.00019


In [4]:
#(schema.jobs & "table_name='__leftover_compartment_final'").delete()

In [5]:
#############################################################################################################

def generate_neighborhood(triangles, num_vertices):
    neighborhood = dict()
    for i in range(num_vertices):
        neighborhood[i] = set()
    for node1, node2, node3 in triangles:
        neighborhood[node1].update([node2, node3])
        neighborhood[node2].update([node1, node3])
        neighborhood[node3].update([node1, node2])
    return neighborhood

def compress_compartments(neighborhood, vertex_labels):
    boundary_clusters = dict()
    for unique_label in np.unique(vertex_labels):
        boundary_clusters[unique_label] = dict()#list()

    starting_node = 0 # This assumes that there are no disconnected portions... I should actually figure out exactly what's going on here.
    visited_nodes = set()
    temp_stack = set()
    temp_stack.add(starting_node)    
    while len(temp_stack) > 0:
        starting_node = temp_stack.pop()
        if starting_node not in visited_nodes:
            same_label_neighbors = set()
            node_label = vertex_labels[starting_node]
            is_on_boundary = False
            for neighboring_node in neighborhood[starting_node]: # Think about if I truly need the same labeled neighbors...
                                                                 # Only way for it to be truly self contained right?
                if node_label == vertex_labels[neighboring_node]:
                    same_label_neighbors.add(neighboring_node)
                else:
                    is_on_boundary = True
            if is_on_boundary:
#                 boundary_clusters[node_label].append((starting_node, same_label_neighbors))
                boundary_clusters[node_label][starting_node] = same_label_neighbors
                
            visited_nodes.add(starting_node)
            temp_stack.update(neighborhood[starting_node])
    return boundary_clusters

def _separate_compartment(neighborhood, cluster, boundary_points):
    components = dict()
    compartment_index = 0
    while len(cluster) > 0:
        visited_nodes = set()
        temp_stack = set()
        temp_stack.add(next(iter(cluster)))
        boundaries_hit = set()
        while len(temp_stack) > 0:
            starting_node = temp_stack.pop()
            if starting_node not in visited_nodes:
                visited_nodes.add(starting_node)
                if starting_node in boundary_points:
                    boundaries_hit.add(starting_node)
                    temp_stack.update(cluster[starting_node])
                else:
                    temp_stack.update(neighborhood[starting_node])
        [cluster.pop(boundary_hit) for boundary_hit in boundaries_hit]        
        components[compartment_index] = visited_nodes
        compartment_index += 1
    return components

def separate_compartments(neighborhood, boundary_clusters):
    compartment_components = dict()
    boundary_clusters_copy = boundary_clusters.copy()
    for label, boundary_cluster in boundary_clusters_copy.items():
        cluster = dict()
        boundary_points = set()
        for node, neighbors in boundary_cluster.items():
            boundary_points.add(node)
            cluster[node] = neighbors
        components = _separate_compartment(neighborhood, cluster, boundary_points)
        compartment_components[label] = components
    return compartment_components
        
############################################################################################################# For Below

@schema
class LeftoverCompartmentFinal(dj.Computed):
    definition = """
    -> pinky.ExcitatoryLeftoverMeshes
    ---
    """

    class LeftoverComponentFinal(dj.Part):
        definition = """
        -> LeftoverCompartmentFinal
        compartment_type   : varchar(16)        # Basal, Apical, spine head, etc.
        component_index    : smallint unsigned  # Which sub-compartment of a certain label this is.
        ---
        n_vertex_indices   : bigint
        n_triangle_indices : bigint
        vertex_indices     : longblob           # preserved indices of each vertex of this sub-compartment
        triangle_indices   : longblob           # preserved indices of each triangle of this sub-compartment
        """
    
    key_source = pinky.ExcitatoryLeftoverMeshes & pinky.CurrentSegmentation & 'decimation_ratio=0.35' & pinky.LeftoverCoarseLabelFinal.proj() & "n_triangles>0"

    
    def make(self, key):
        def generate_triangle_neighborhood(triangles):
            """
            Maps each vertex node to every triangle they appear in.
            """
            triangle_neighborhood = dict()
            for i in range(len(triangles)):
                triangle_neighborhood[i] = set()
            for i, (node1, node2, node3) in enumerate(triangles):
                triangle_neighborhood[node1].add(i)
                triangle_neighborhood[node2].add(i)
                triangle_neighborhood[node3].add(i)
            return triangle_neighborhood
        
        def generate_component_keys(key, components, triangles, triangle_neighborhood, labeled_triangles):
            for label_key, compartment in components.items():
                for component_index, component in compartment.items():
                    try:
                        label_name = (pinky.LabelKey & dict(numeric=label_key)).fetch1('description')
                    except:
                        label_name = str(label_key)
                        
                    vertex_indices = np.array(list(component))
                    triangle_indices = np.unique([triangle_index for node in component
                                                  for triangle_index in triangle_neighborhood[node]
                                                  if labeled_triangles[triangle_index] == label_key])
                    set_vertex_indices = set(vertex_indices)
                    true_triangle_indices = []
                    for triangle_index in triangle_indices:
                        node1, node2, node3 = triangles[triangle_index]
                        if node1 in set_vertex_indices:
                            if node2 in set_vertex_indices:
                                if node3 in set_vertex_indices:
                                    true_triangle_indices.append(triangle_index)                        
                    triangle_indices = np.array(true_triangle_indices)
                    yield dict(key,
                               compartment_type=label_name,
                               component_index=component_index,
                               n_vertex_indices=len(vertex_indices),
                               n_triangle_indices=len(triangle_indices),
                               vertex_indices=vertex_indices,
                               triangle_indices=triangle_indices)
        
        start = time.time()
        #print("hello")
        mesh = (pinky.ExcitatoryLeftoverMeshes & key).fetch1()
        labels = (pinky.LeftoverCoarseLabelFinal & key).fetch1()
        #print("something")
        if len(np.unique(labels['triangles'])) == 1:
            #print("heyo")
            self.insert1(key)
            label_name = (pinky.LabelKey & dict(numeric=np.unique(labels['triangles'])[0])).fetch1('description')
            vertex_indices = np.arange(len(labels['vertices']), dtype=np.uint32)
            triangle_indices = np.arange(len(labels['triangles']), dtype=np.uint32)
            new_dict= dict(key,
                                                compartment_type=label_name,
                                                component_index=0,
                                                n_vertex_indices=len(vertex_indices),
                                                n_triangle_indices=len(triangle_indices),
                                                vertex_indices=vertex_indices,
                                                triangle_indices=triangle_indices)
            
            LeftoverCompartmentFinal.LeftoverComponentFinal().insert1(dict(key,
                                                compartment_type=label_name,
                                                component_index=0,
                                                n_vertex_indices=len(vertex_indices),
                                                n_triangle_indices=len(triangle_indices),
                                                vertex_indices=vertex_indices,
                                                triangle_indices=triangle_indices))
            return
        
        neighborhood = generate_neighborhood(mesh['triangles'], len(mesh['vertices']))
        boundary_clusters = compress_compartments(neighborhood, labels['vertices'])
        components = separate_compartments(neighborhood, boundary_clusters)
        triangle_neighborhood = generate_triangle_neighborhood(mesh['triangles'])

        self.insert1(key)
        LeftoverCompartmentFinal.LeftoverComponentFinal().insert(generate_component_keys(key, components, mesh['triangles'],
                                                               triangle_neighborhood, labels['triangles']))

        print(key['segment_id'], "finished separating components:", time.time() - start)

In [6]:
start_time = time.time()
LeftoverCompartmentFinal.populate(reserve_jobs=True)
print(f"Total time = {time.time() - start_time}")

648518346349495971 finished separating components: 0.6883349418640137
648518346349496058 finished separating components: 0.7780566215515137
648518346349496405 finished separating components: 7.399611473083496
648518346349498482 finished separating components: 0.6339643001556396
648518346349498566 finished separating components: 0.1761476993560791
648518346349498835 finished separating components: 1.2366180419921875
648518346349499636 finished separating components: 0.267000675201416
648518346349499680 finished separating components: 0.1031179428100586
648518346349499689 finished separating components: 0.3431060314178467
648518346349499773 finished separating components: 0.20007824897766113
648518346349499803 finished separating components: 0.2628648281097412
648518346349499828 finished separating components: 0.37020230293273926
648518346349499896 finished separating components: 0.6770720481872559
648518346349500138 finished separating components: 0.8582801818847656
648518346349500431 f

# check that all neurons have components

In [7]:
#(schema.jobs & "table_name='__compartment_final'").delete()

In [8]:
# #check that there are all components in there
# pinky.CompartmentFinal.ComponentFinal()

In [9]:
# segment_ids = pinky.CompartmentFinal.ComponentFinal.fetch("segment_id")
# segment_ids

In [10]:
# from collections import Counter
# segment_id_counters = Counter(segment_ids)
# #segment_id_counters

In [11]:
# for i in segment_id_counters.values():
#     if i == 0:
#         print("0 value")

In [12]:
# print(len(segment_id_counters.keys()))
# print(np.sum(np.where(segment_id_counters.values() > 0)))

In [13]:
"""
Conclusion: There were all segments present and no segments that had 0 parts
So it is ready for spine computation because all have passed


"""

'\nConclusion: There were all segments present and no segments that had 0 parts\nSo it is ready for spine computation because all have passed\n\n\n'